# Note
*Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

*Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import ipywidgets as widgets
from IPython.display import display


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'CityWeather.csv')

city_df = pd.read_csv(csv_file)
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Yellowknife,20,CA,1616149752,85,62.4560,-114.3525,12.20,9.22
1,Aplahoué,100,BJ,1616150073,47,6.9333,1.6833,91.71,5.37
2,Okhotsk,0,RU,1616149948,68,59.3833,143.3000,7.81,13.60
3,Klaksvík,73,FO,1616149762,93,62.2266,-6.5890,46.40,9.22
4,São Félix do Xingu,98,BR,1616150040,97,-6.6447,-51.9950,71.76,2.62
...,...,...,...,...,...,...,...,...,...
542,Lalibela,0,ET,1616150223,16,12.0333,39.0333,80.24,3.69
543,Churapcha,100,RU,1616150223,95,62.0025,132.4325,10.49,7.29
544,Black River,20,JM,1616150223,78,18.0264,-77.8487,73.40,3.44
545,Guerrero Negro,35,MX,1616149755,85,27.9769,-114.0611,55.92,9.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(int)

# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = city_df.loc[(city_df["Max Temp"] > 60) & (city_df["Max Temp"] < 80) & (city_df["Cloudiness"] == 0), :]

# Drop any rows will null values.
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Lincoln,0,AU,1616149761,71,-34.7333,135.8667,68.70,12.35
1,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05
2,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75
3,Lázaro Cárdenas,0,MX,1616149882,76,17.9583,-102.2000,69.67,4.99
4,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=narrowed_city_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Port Lincoln,0,AU,1616149761,71,-34.7333,135.8667,68.70,12.35,
1,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05,
2,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75,
3,Lázaro Cárdenas,0,MX,1616149882,76,17.9583,-102.2000,69.67,4.99,
4,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22,
5,Kadoma,0,JP,1616150002,31,34.7333,135.5833,62.60,12.66,
6,Esperance,0,AU,1616149760,53,-33.8667,121.9000,75.20,13.80,
7,New Norfolk,0,AU,1616149762,61,-42.7826,147.0587,62.01,1.01,
8,Arraial do Cabo,0,BR,1616149748,94,-22.9661,-42.0278,75.20,4.61,
9,Lima,0,PE,1616150079,83,-12.0432,-77.0282,69.80,5.75,


In [6]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        
        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Port Lincoln,0,AU,1616149761,71,-34.7333,135.8667,68.70,12.35,Port Lincoln Hotel
1,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05,Villas Carrizalillo Hotel
2,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75,Raoum Inn Hotel
3,Lázaro Cárdenas,0,MX,1616149882,76,17.9583,-102.2000,69.67,4.99,City Express Lázaro Cárdenas
4,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22,The Pilot House - Inn Between the Beaches
5,Kadoma,0,JP,1616150002,31,34.7333,135.5833,62.60,12.66,Hotel New Otani Osaka
6,Esperance,0,AU,1616149760,53,-33.8667,121.9000,75.20,13.80,Esperance Island View Apartments
7,New Norfolk,0,AU,1616149762,61,-42.7826,147.0587,62.01,1.01,Tynwald Willow Bend Estate
8,Arraial do Cabo,0,BR,1616149748,94,-22.9661,-42.0278,75.20,4.61,Orlanova Boutique Hotel
9,Lima,0,PE,1616150079,83,-12.0432,-77.0282,69.80,5.75,Sheraton Lima Hotel & Convention Center


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer and info box content ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))